# Estrattore degli appalti da remoto a locale

In [40]:
# Import
import pandas as pd
import numpy as np
import matplotlib
import logging
import urllib
import io
import csv
import time
import os
from lxml import etree

# Constraints
APPALTI_XSD = "http://dati.anticorruzione.it/schema/datasetAppaltiL190.xsd"
INDICE_APPALTI_XSD = "http://dati.anticorruzione.it/schema/datasetIndiceAppaltiL190.xsd"

In [4]:
!head -2 ../data/com_suc.csv

"codiceFiscale";"dataUltimoTentativoAccessoUrl";"esitoUltimoTentativoAccessoUrl";"identificativoPEC";"ragioneSociale";"url"
"84002280547";"2015-04-02T15:21:41.117+0000";"successo";"opec275.20150110123012.28213.03.1.5@pec.actalis.it";"ISTITUTO COMPRENSIVO ""G.G. PONTANO"" CERRETO DI SPOLETO";"http://omnicomprensivocerretodispoleto.it/avcp/trasparenza/11/2014.xml"


In [9]:
# Carica comunicazioni avvenute con successo
com_suc = pd.read_csv('../data/com_suc.csv', sep=';', quoting=csv.QUOTE_ALL)

print ("Numero di url su cui iterare: ",com_suc.shape[0])

Numero di url su cui iterare:  34544


In [46]:
# Ricavare lo schema degli appalti e indice
appalti_xmlschema = etree.XMLSchema(etree.parse(APPALTI_XSD))
indice_xmlschema = etree.XMLSchema(etree.parse(INDICE_APPALTI_XSD))

# UnComment for test
#comunicazioni.drop(comunicazioni.loc[50:].index, inplace=True);

In [87]:
def get_log(name, force_init=False):
    logger = logging.getLogger(name)
    logger.setLevel(logging.DEBUG)

    if force_init==True:
        
        # remove all handler if exist
        for i,v in enumerate(logger.handlers):
            logger.removeHandler(logger.handlers[i])
        
        # delete file log if exist
        try:
            os.remove(name)
        except OSError:
            pass
        
        # create a file handler
        handler = logging.FileHandler(name)
        handler.setLevel(logging.DEBUG)

        # create a logging format
        formatter = logging.Formatter('%(asctime)s;%(name)s;%(levelname)s;%(message)s')
        handler.setFormatter(formatter)

        # add the handlers to the logger
        logger.addHandler(handler)
        
    return logger

In [93]:
# Inizializza logger
logger = get_log(name='e02-appalti-remote2local.log', force_init=True)

In [ ]:
#%%timeit

logger.info("=== Start ===")
#url=['../data/test/acqueblspa_legge190_2014.xml', '../data/test/Cittadella-contratti-2014.xml', '../data/test/dataset1.xml','../data/test/sangimignano.xml', '../data/test/test.xml']
counter_so_far, tid, rid, arid=0,0,0,0

# Per ogni url 
# ... Prova a scaricare l'xml se va in errore scrivi in log l'errore
# ... se lo schema è appalti trasforma in csv e scrivi in log
# ... se lo schema è indice scrivi in log
# ... altrimenti logga xml schema sconosciuto
for u in url_np[0:10000]:
#for u in url:
    time.sleep( 1 )
    try:
        doc = etree.parse(u)
        etree.strip_tags(doc,etree.Comment)
        if appalti_xmlschema.validate(doc):
            #tid, rid, arid=appalti_xml2csv(doc, counter_so_far, tid, rid, arid)
            # Save doc as counter_so_far.xml
            doc.write('../data/appalti_xml/{}.xml'.format(counter_so_far))
            logger.info(";%d;appalti;fatto" %(counter_so_far))
        elif indice_xmlschema.validate(doc):
            logger.info(";%d;indice;saltato" %(counter_so_far))    
        else:
            logger.info(";%d;schema_errato;saltato" %(counter_so_far))
    except (urllib.error.URLError, urllib.error.HTTPError, OSError, etree.XMLSyntaxError) as e:
        logger.info(";%d;errore:%s;saltato" %(counter_so_far, type(e).__name__))
 
    counter_so_far +=1